### Investigando con H2O y automodel. 
He realizado diferentes pruebas con los datos, y eliminando columnas. He conseguido "la mejor predicción" tal y como está ahora el notebook.

In [58]:
import pandas as pd
import h2o
from h2o.automl import H2OAutoML

In [59]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
testid = test['id']

In [60]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O cluster uptime:,1 hour 42 mins
H2O cluster timezone:,Europe/Madrid
H2O data parsing timezone:,UTC
H2O cluster version:,3.26.0.2
H2O cluster version age:,7 months and 4 days !!!
H2O cluster name:,H2O_from_python_soyungalgo_jxip91
H2O cluster total nodes:,1
H2O cluster free memory:,1.807 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


In [44]:
h2train = h2o.H2OFrame(train)
h2test = h2o.H2OFrame(test)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [45]:
columnas = [a for a in h2train.columns if a != "price"]
x = columnas
y = "price"
print(columnas)

['id', 'carat', 'depth', 'table', 'x', 'y', 'z', 'cut_Fair', 'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D', 'color_E', 'color_F', 'color_G', 'color_H', 'color_I', 'color_J', 'clarity_I1', 'clarity_IF', 'clarity_SI1', 'clarity_SI2', 'clarity_VS1', 'clarity_VS2', 'clarity_VVS1', 'clarity_VVS2']


In [46]:
#H2oAutoML buscará automáticamente modelos de machine learning que se ajustan a los datos y entrenaremos con el mejor

In [47]:
automl = H2OAutoML(max_models=50, seed=42,max_runtime_secs=300, sort_metric='RMSE')
automl.train(x=x, y=y, training_frame=h2train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [48]:
print('[INFO] Models leader board:')
leader_board = automl.leaderboard
leader_board.head()

[INFO] Models leader board:


model_id,mean_residual_deviance,rmse,mse,mae,rmsle
XGBoost_1_AutoML_20200302_123930,304094,551.447,304094,277.407,0.0964602
XGBoost_2_AutoML_20200302_123930,355539,596.271,355539,300.286,0.0960263
StackedEnsemble_AllModels_AutoML_20200302_123930,7.03234e+06,2651.86,7.03234e+06,1983.41,0.854323
StackedEnsemble_BestOfFamily_AutoML_20200302_123930,1.11386e+07,3337.46,1.11386e+07,2527.34,0.995777
XGBoost_3_AutoML_20200302_123930,1.36881e+07,3699.74,1.36881e+07,2437.73,1.08642


In [49]:
metrics = automl.leader.cross_validation_metrics_summary().as_data_frame()
metrics

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,mae,277.40747,4.94194,271.06802,284.25885,275.24405,286.92386,269.54263
1,mean_residual_deviance,304094.25,13254.099,305579.62,330176.97,299363.38,312563.34,272787.84
2,mse,304094.25,13254.099,305579.62,330176.97,299363.38,312563.34,272787.84
3,r2,0.98094124,5.779693E-4,0.98042154,0.98055583,0.9804996,0.9806608,0.98256856
4,residual_deviance,304094.25,13254.099,305579.62,330176.97,299363.38,312563.34,272787.84
5,rmse,551.1817,12.102796,552.79254,574.6103,547.1411,559.07367,522.29095
6,rmsle,0.0964554,6.8277546E-4,0.097121775,0.09658108,0.096728854,0.097256735,0.09458857


In [50]:
automl.leader.varimp(use_pandas=True)

,variable,relative_importance,scaled_importance,percentage
0,y,1.480187e+12,1.000000,0.378016
1,carat,1.475907e+12,0.997108,0.376923
2,x,4.842036e+11,0.327123,0.123658
3,z,9.458418e+10,0.063900,0.024155
4,clarity_SI2,6.237338e+10,0.042139,0.015929
5,clarity_I1,5.112789e+10,0.034541,0.013057
6,color_J,4.004647e+10,0.027055,0.010227
7,clarity_SI1,3.284784e+10,0.022192,0.008389
8,clarity_VVS2,2.943144e+10,0.019884,0.007516
9,color_I,2.783098e+10,0.018802,0.007108


In [51]:
predictions = automl.leader.predict(h2test)

xgboost prediction progress: |████████████████████████████████████████████| 100%


In [52]:
test["price"] = predictions.as_data_frame()
columnasentrega = ["id","price"]
entrega = test["price"]
test = test[columnasentrega]

In [54]:
test.head()

,id,price
0,0,4432.266113
1,1,1060.938354
2,2,17236.824219
3,3,7067.860352
4,4,1468.447754


In [55]:
test.to_csv('output/predictfinal2.csv', index=False)

In [56]:
#mejor_Xgboost_elim_xyz = h2o.save_model(model=automl.leader, path="src/mejor_Xgboost_elim_xyz.h2o", force=True)

In [57]:
testing = pd.read_csv('output/predictfinal.csv')

In [1]:
h2o.cluster().shutdown()

NameError: name 'h2o' is not defined